In [1]:
from dotenv import load_dotenv
load_dotenv()
import os, json, pickle, time
import regex as re
from bot import set_driver, open_page, get_href_page, check_page
from parse import get_article_info

output_name = "gal767"

input_company_file = "STEP4_Factiva_company_list_No5301_5350_sector_Ishika.csv"

ut_eid = "gal767"

eid_password = os.getenv('eid_password')

driver_path = "C://Users//galon//cd_secure//chromedriver.exe"

ARTICLE_DIR = os.path.join('..//data', 'article_hrefs', 'test')

os.makedirs(ARTICLE_DIR, exist_ok=True)

In [2]:
def gen_searches():
    
    try:
        with open('step5_searches.pickle', 'rb') as file:
            articles = pickle.load(file)
            if len(articles) == 0:
                print('Create a new pickle to manage searches; current list has been exhausted.')
    except FileNotFoundError:
        articles = {}
        for file_name in os.listdir(ARTICLE_DIR):
            l = []
            with open(os.path.join(os.path.join(ARTICLE_DIR, file_name))) as file:
                href_list =  [t for t in re.sub('\n','',file.read()).split(r'"') if len(re.findall('\w+', t)) > 0]
                for href in href_list:
                    l.append(f"https://global-factiva-com.ezproxy.lib.utexas.edu/{href.split('&')[0].split('../')[1]}")
            articles[file_name.split('.')[0]] = l
    return articles

In [5]:
def get_text(eid_username, eid_password, path):
    
    searches = gen_searches()
    searches_pickle = searches.copy()
    driver, wait = set_driver(path)
    open_page(driver, wait, eid_username, eid_password)
    print(len(searches))
    
    for name, list_ in searches.items():
        print(list_)
        temp = {name: {}}
        for n, href in enumerate(list_):
            print(href)
            temp[name][n] = {}
            if "/du/article.aspx/?accessionno=J" in href:
                print(href)
                get_href_page(driver, wait, href)
                check_page(driver, wait, href)
                text_dict = get_article_info(driver.page_source, temp[name][n])
                time.sleep(5)
        try:
            searches_pickle.pop(name)
            print(text_dict)
            with open('step5_searches.pickle', 'wb') as file:
                pickle.dump(searches_pickle, file)
            with open('test_dict.json', 'a+') as file:
                file.write('\n')
                json.dump(text_dict, file)
                file.write('\n')
        except:
            return 'Error when saving'
    return text_dict

In [6]:
get_text(ut_eid, eid_password, driver_path)

2
['https://global-factiva-com.ezproxy.lib.utexas.edu/du/article.aspx/?accessionno=j000000020021018dyai00009']
https://global-factiva-com.ezproxy.lib.utexas.edu/du/article.aspx/?accessionno=j000000020021018dyai00009


'Error when saving'